In [2]:

import requests


In [2]:
#What are the chances gpt will understand this?
test_users = """\
| Username    | Password          | Role  |
|-------------|-------------------|-------|
| john.doe    | strong-password   | User  |
| jane.smith  | you-dont-guess-me | User  |
| jimmy.allen | secure-secret     | Admin |
"""

In [2]:
app_under_testing_path = "../postfinance/source"
yaml_path = app_under_testing_path + "/src/main/resources/openapi/openapi.yml"
requirements_path = app_under_testing_path + "/spec/Requirements.md"

In [21]:
#TODO search common endpoints for yaml
with open(requirements_path, 'r') as file:
    stories = file.read().split("\n## ")[1:]
    stories = [[y for y in x.split("\n### Scenario:") if y != ""] for x in stories]
    for story in stories:
        print("\n".join(story))

User Story 1: Customer Registration
As a new customer, I want to register for an online banking account so that I can access banking services.


- **Given** I am not registered to the online banking service
- **When** I navigate to the bank's registration page
- **And** submit my firstname, lastname and birthdate
- **Then** I receive confirmation of successful registration in form of a username and password

User Story 2: Customer Login
As a registered customer, I want to be able to log into my online banking account.


- **Given** I am registered to the online banking service
- **When** I navigate to the bank's login page
- **And** submit a valid combination of username and password
- **Then** I am logged in
- **And** being redirected to my homepage

User Story 3: Account Balance Check
As a customer, I want to check my account balance online to monitor my finances.


- **Given** I am logged into my online banking account
- **When** I am visiting my homepage
- **Then** I should see a l

In [12]:

import requests
import json

def test_customer_registration_success():
    # Set up the API endpoint and test data
    url = "http://localhost:8080/customers/register"
    headers = {"Content-Type": "application/json"}

    # Use valid user credentials from the table
    data = json.dumps({
        "firstName": "John",
        "lastName": "Doe",
        "dateOfBirth": "1990-01-01"
    })

    # Send the registration request
    response = requests.post(url, headers=headers, data=data, timeout=5)

    # Assert that the API returns a 201 Created status code
    assert response.status_code == 201

    # Parse the response JSON
    response_data = json.loads(response.text)

    # Assert that the response includes a username and password
    assert "username" in response_data
    assert "password" in response_data

    # Assert that the username is a combination of the first name and last name
    assert response_data["username"] == "john.doe"

test_customer_registration_success()

500
500


AssertionError: 

In [5]:

import requests
import json

def test_customer_registration_exceeds_max_length():
    # Set up the API endpoint and test data
    url = "http://localhost:8080/customers/register"
    headers = {"Content-Type": "application/json"}

    # Test first name exceeds max length
    data = json.dumps({
        "firstName": "a" * 33,
        "lastName": "Doe",
        "dateOfBirth": "1990-01-01"
    })
    response = requests.post(url, headers=headers, data=data)
    assert response.status_code == 400

    # Test last name exceeds max length
    data = json.dumps({
        "firstName": "John",
        "lastName": "a" * 33,
        "dateOfBirth": "1990-01-01"
    })
    response = requests.post(url, headers=headers, data=data)
    assert response.status_code == 400

test_customer_registration_exceeds_max_length()

AssertionError: 

# Pseudocode for prompt enginering

`k` number of test cases to generate (for quorum, trying to avoid hallucinations)

`T` temperature param of generation

        parse user stories into a list
        for each user story:
          ask gpt for test ideas
          CONTEXT: in context, formatted examples of test ideas for stories
          for each test idea:
              ask gpt k times for the test # k = 1 if T = 0
              CONTEXT: default user definition, openAPI spec, in context examples
        run tests
        if number_successes == k: pass ✅
        if number_successes == 0: fail ❌
        otherwise: gpt error 🤯


nice to have: detect syntax errors in running tests and report gpt failure


